In [1]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, InputLayer
from keras.layers import SeparableConv1D
import statistics
from statistics import mode
import random
import os

import math

import pylab as p
import seaborn as sns
from datetime import datetime, timedelta
import string
from sklearn.preprocessing import OneHotEncoder

# Read and process data

### Calculate the power and put it in a dataframe

In [2]:
def concatenate_files_in_folders(base_path, folder_prefix):
    # Initialize an empty list to store file location paths
    sources = list()
    
    def recursive_search(base_path):
        for root, dirs, files in os.walk(base_path):
            for dir in dirs:
                if dir.startswith(folder_prefix):
                    folder_path = os.path.join(root, dir)
                    sources.append(folder_path)
                else:
                    recursive_search(os.path.join(root, dir))
                    break # TODO: Remove to work with all houses (not just H1)
            break

    recursive_search(base_path)
    
    return sources

In [3]:
base_path = ".\dataset"
train_folder_prefix = "Tagged_Training_"
test_folder_prefix = "Testing_"

file_path_volts1 = r"LF1V.csv"
file_path_amps1 = r"LF1I.csv"
file_path_time_ticks1 = r"TimeTicks1.csv"

file_path_volts2 = r"LF2V.csv"
file_path_amps2 = r"LF2I.csv"
file_path_time_ticks2 = r"TimeTicks2.csv"

file_path_tagging_info = r".\dataset\CompleteTaggingInfo.csv"
file_path_tagging_info_generalized = r".\dataset\AllTaggingInfo_generalized.csv"

sources = concatenate_files_in_folders(base_path, train_folder_prefix)

for source in sources:
    LF1V = pd.read_csv(os.path.join(source, file_path_volts1))
    LF1I = pd.read_csv(os.path.join(source, file_path_amps1))
    time_ticks1 = pd.read_csv(os.path.join(source, file_path_time_ticks1))
    LF2V = pd.read_csv(os.path.join(source, file_path_volts2))
    LF2I = pd.read_csv(os.path.join(source, file_path_amps2))
    time_ticks2 = pd.read_csv(os.path.join(source, file_path_time_ticks2))
    training_set = calculate_power(LF1V, LF1I, time_ticks1, LF2V, LF2I, time_ticks2)

tagging_info = pd.read_csv(file_path_tagging_info)
general_tagging_info = pd.read_csv(file_path_tagging_info_generalized)

In [4]:
def label_generalization(ids, general_labels):
    generalized_ids = np.empty(0)
    for id in ids:
        generalized_ids = np.append(generalized_ids, general_labels.loc[general_labels['ApplianceID'] == id, 'GeneralID'].unique()[0])
    return generalized_ids

In [5]:
print(label_generalization([1, 2, 3], general_tagging_info))

.\dataset\H1\Tagged_Training_04_13_1334300401: 
.\dataset\H1\Tagged_Training_10_22_1350889201: 
.\dataset\H1\Tagged_Training_10_23_1350975601: 
.\dataset\H1\Tagged_Training_10_24_1351062001: 
.\dataset\H1\Tagged_Training_10_25_1351148401: 
.\dataset\H1\Tagged_Training_12_27_1356595201: 
.\dataset\H2\Tagged_Training_02_15_1360915201: 
.\dataset\H2\Tagged_Training_06_13_1339570801: 
.\dataset\H2\Tagged_Training_06_14_1339657201: 
.\dataset\H2\Tagged_Training_06_15_1339743601: 
.\dataset\H3\Tagged_Training_07_30_1343631601: 
.\dataset\H3\Tagged_Training_07_31_1343718001: 
.\dataset\H3\Tagged_Training_08_01_1343804401: 
.\dataset\H4\Tagged_Training_07_26_1343286001: 
.\dataset\H4\Tagged_Training_07_27_1343372401: 


In [6]:
labels = np.empty(0)
labels = np.append(labels, 1)
labels = np.append(labels, 2)
labels = np.append(labels, 3)
encoder = OneHotEncoder(sparse_output=False, categories=[list(range(19))])
labels_encoded = encoder.fit_transform(np.array(labels).reshape(-1, 1))

[        Phase1_actual  Phase1_real  Phase1_img  Phase1_app   Phase1_time  \
0           19.430857    24.264125   18.475752   30.497560  1.334300e+09   
1           19.924720    24.357135   17.474827   29.977318  1.334300e+09   
2           19.690603    24.166927   17.523352   29.851436  1.334300e+09   
3           19.910825    24.234800   17.153411   29.691161  1.334300e+09   
4           20.829979    25.187723   17.446867   30.640081  1.334300e+09   
...               ...          ...         ...         ...           ...   
518938      19.939808    24.472770   17.732765   30.221969  1.334387e+09   
518939      19.792885    24.371436   17.809936   30.185439  1.334387e+09   
518940      20.446493    24.783498   17.294649   30.221295  1.334387e+09   
518941      20.235556    24.840667   18.001915   30.677804  1.334387e+09   
518942      19.618725    24.178684   17.711506   29.971757  1.334387e+09   

        Phase2_actual  Phase2_real  Phase2_img  Phase2_app   Phase2_time  
0          

In [7]:
def calculate_power(LF1V, LF1I, TimeTicks1, LF2V, LF2I, TimeTicks2):
    LF1V = np.array(LF1V)
    LF1I = np.array(LF1I)
    LF1V = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF1V])
    LF1I = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF1I])

    LF2V = np.array(LF2V)
    LF2I = np.array(LF2I)
    LF2V = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF2V])
    LF2I = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF2I])

    L1_P = LF1V * np.conjugate(LF1I)
    L2_P = LF2V * np.conjugate(LF2I)

    L1_ComplexPower = np.sum(L1_P, axis=1)
    L2_ComplexPower = np.sum(L2_P, axis=1)

    L1_real = np.real(L1_ComplexPower)
    L1_imag = np.imag(L1_ComplexPower)
    L1_app = np.abs(L1_ComplexPower)

    L2_real = np.real(L2_ComplexPower)
    L2_imag = np.imag(L2_ComplexPower)
    L2_app = np.abs(L2_ComplexPower)

    L1_Pf = np.cos(np.angle(L1_P[:, 0]))
    L2_Pf = np.cos(np.angle(L2_P[:, 0]))

    L1_actual_power = L1_real * L1_Pf
    L2_actual_power = L2_real * L2_Pf

    time_ticks1 = np.array(TimeTicks1)
    time_ticks2 = np.array(TimeTicks2)

    data = pd.DataFrame()
    len = min(L1_actual_power.size, L2_actual_power.size)

    data["Phase1_actual"] = L1_actual_power[:len]
    data["Phase1_real"] = L1_real[:len]
    data["Phase1_img"] = L1_imag[:len]
    data["Phase1_app"] = L1_app[:len]
    data["Phase1_time"] = time_ticks1[:len]

    data["Phase2_actual"] = L2_actual_power[:len]
    data["Phase2_real"] = L2_real[:len]
    data["Phase2_img"] = L2_imag[:len]
    data["Phase2_app"] = L2_app[:len]
    data["Phase2_time"] = time_ticks2[:len]

    return data

### Go through directory and process the days

In [8]:
# Directory is the path to the directory where to house's information is
def read_files_in_directory(directory):
    all_tagging_info = pd.DataFrame()
    labels = []
    data = []

    for root, _, files in os.walk(directory):
        if "Testing" in root:
            continue

        LF1I = pd.DataFrame()
        LF1V = pd.DataFrame()
        TimeTicks1 = pd.DataFrame()
        LF2I = pd.DataFrame()
        LF2V = pd.DataFrame()
        TimeTicks2 = pd.DataFrame()
        for file in files:

            file_path = os.path.join(root, file)
            if file == "CompleteTaggingInfo.csv":
                all_tagging_info = pd.read_csv(file_path)
                continue

            if file == "TaggingInfo.csv":
                # labels.append(pd.read_csv(file_path))
                continue

            if file == "LF1I.csv":
                LF1I = pd.read_csv(file_path)
                continue

            if file == "LF1V.csv":
                LF1V = pd.read_csv(file_path)
                continue

            if file == "TimeTicks1.csv":
                TimeTicks1 = pd.read_csv(file_path)
                continue

            if file == "LF2I.csv":
                LF2I = pd.read_csv(file_path)
                continue

            if file == "LF2V.csv":
                LF2V = pd.read_csv(file_path)
                continue

            if file == "TimeTicks2.csv":
                TimeTicks2 = pd.read_csv(file_path)
                continue

        if not LF1I.empty and not LF1V.empty and not LF2I.empty and not LF2V.empty:
            print(f"{root}: ")
            day_data = calculate_power(LF1I, LF1V, TimeTicks1, LF2I, LF2V, TimeTicks2)
            data.append(day_data)

    return data, labels, all_tagging_info

In [9]:
data, _, all_tagging_info = read_files_in_directory(r".\dataset\H1")

In [10]:
print(data)

# Next we need to detect all labeled events

### All the methods needed to detect, filter and extract spikes

In [14]:
def moving_average(array, window):
    moving_avg = np.convolve(array, np.ones(window) / window, mode='valid')
    return moving_avg


def normalize(array):
    min_val = np.min(array)
    max_val = np.max(array)
    normalized_array = (array - min_val) / (max_val - min_val)
    return normalized_array


def detect_cusum(array, threshold, drift):
    sum_positive = np.zeros(array.size)
    sum_negative = np.zeros(array.size)
    event_index_start = np.array([[], [], []], dtype=int)
    event_index_end = np.array([[], [], []], dtype=int)

    for i in range(1, array.size):
        sum = array[i] - array[i - 1]

        # sums for positive/negative changes
        sum_positive[i] = max(0, sum_positive[i - 1] + sum - drift)
        sum_negative[i] = max(0, sum_negative[i - 1] - sum - drift)

        # if a change is detected
        if sum_positive[i] > threshold:
            event_index_start = np.append(event_index_start, i)
            sum_positive[i], sum_negative[i] = 0, 0
            continue

        if sum_negative[i] > threshold:
            event_index_end = np.append(event_index_end, i)
            sum_positive[i], sum_negative[i] = 0, 0

    return event_index_start, event_index_end


def filter_close_events(event_indices, min_separation):
    filtered_indices = [event_indices[0]]

    for i in range(1, event_indices.size):
        if event_indices[i] - event_indices[i - 1] >= min_separation:
            # If the time separation is greater than or equal to the threshold, keep the event
            filtered_indices.append(event_indices[i])

    return np.array(filtered_indices)


# Short spikes in power when an appliance starts up can be detected as an end event, so make sure that start and end events are not too close together
def filter_spikes(events_start, events_end, min_separation):
    indexes_to_delete = list()

    for iter in range(4):
        for i in range(events_start.size):
            for j in range(events_end.size):
                if events_end[j] < events_start[i]:
                    continue

                if events_end[j] - events_start[i] < min_separation:
                    indexes_to_delete.append(j)
                    break

                break

        events_end = np.delete(events_end, indexes_to_delete)
        indexes_to_delete = list()
    return events_end


def group_start_end(events_start, events_end):
    start_indexes = list()
    end_indexes = list()

    for i in range(events_start.size):
        for j in range(events_end.size):
            if events_end[j] < events_start[i]:
                continue

            start_indexes.append(events_start[i])
            end_indexes.append(events_end[j])
            break

    return np.array(start_indexes), np.array(end_indexes)


def filter_short_events(events_start, events_end, min_event_length):
    start_indexes = list()
    end_indexes = list()

    for i in range(events_start.size):
        if events_end[i] - events_start[i] >= min_event_length:
            start_indexes.append(events_start[i])
            end_indexes.append(events_end[i])

    return np.array(start_indexes), np.array(end_indexes)


def calculate_overlap(start, end, on, off):
    return (min(end, off) - max(start, on)) / (end - start)


def find_closest_event(events_start, events_end, time_ticks, label):
    start = 0
    end = max(events_end)

    for event in range(events_start.size):
        if events_start[event] > start and time_ticks[int(events_start[event])] <= label["OnTime"]:
            start = events_start[event]

        if events_end[event] < end and time_ticks[int(events_end[event])] >= label["OffTime"]:
            end = events_end[event]

    return start, end


def extract_labeled_spikes(events_start, events_end, time_ticks, labels):
    labeled_events_start = list()
    labeled_events_end = list()
    event_labels = list()

    start_of_day = time_ticks[0][0]
    end_of_day = time_ticks[0][time_ticks[0].size - 1]

    for index in range(labels.shape[0]):
        row = labels.iloc[index]
        found = False

        if not (start_of_day <= row["OnTime"] <= end_of_day):
            continue

        last_event = 0
        phase_index = 0
        for event in range(events_start.size):
            start = int(events_start[event])
            end = int(events_end[event])

            if start < last_event:
                phase_index += 1

            last_event = start

            time = time_ticks[phase_index]

            if not (row["OnTime"] <= time[start] <= row["OffTime"] or row["OnTime"] <= time[end] <= row["OffTime"]):
                continue

            found = True
            overlap = calculate_overlap(time[start], time[end], row["OnTime"], row["OffTime"])
            if overlap < 0.3:
                continue

            labeled_events_start.append(start)
            labeled_events_end.append(end)
            event_labels.append(row["ID"])

        if not found:
            start, end = find_closest_event(events_start, events_end, time_ticks[phase_index], row)
            labeled_events_start.append(start)
            labeled_events_end.append(end)
            event_labels.append(row["ID"])

    return np.array(labeled_events_start), np.array(labeled_events_end), np.array(event_labels)


def calculate_events(phase_power):
    phase_power = moving_average(phase_power, 7)
    phase_power = normalize(phase_power)

    events_start, events_end = detect_cusum(phase_power, threshold=0.009, drift=0.0005)
    events_start = filter_close_events(events_start, 60)
    events_end = filter_close_events(events_end, 60)
    events_end = filter_spikes(events_start, events_end, 12)
    events_start, events_end = group_start_end(events_start, events_end)
    events_start, events_end = filter_short_events(events_start, events_end, 100)

    return events_start, events_end


def remove_overlapping_events(events_start, events_end, event_labels):
    to_remove = list()

    for index in range(1, events_start.size):
        if events_end[index] == events_end[index - 1]:
            to_remove.append(index)

    return np.delete(events_start, to_remove), np.delete(events_end, to_remove), np.delete(event_labels, to_remove)

        Phase1_actual  Phase1_real  Phase1_img  Phase1_app   Phase1_time  \
0           19.430857    24.264125   18.475752   30.497560  1.334300e+09   
1           19.924720    24.357135   17.474827   29.977318  1.334300e+09   
2           19.690603    24.166927   17.523352   29.851436  1.334300e+09   
3           19.910825    24.234800   17.153411   29.691161  1.334300e+09   
4           20.829979    25.187723   17.446867   30.640081  1.334300e+09   
...               ...          ...         ...         ...           ...   
518938      19.939808    24.472770   17.732765   30.221969  1.334387e+09   
518939      19.792885    24.371436   17.809936   30.185439  1.334387e+09   
518940      20.446493    24.783498   17.294649   30.221295  1.334387e+09   
518941      20.235556    24.840667   18.001915   30.677804  1.334387e+09   
518942      19.618725    24.178684   17.711506   29.971757  1.334387e+09   

        Phase2_actual  Phase2_real  Phase2_img  Phase2_app   Phase2_time  \
0          

In [7]:
def label_generalization(ids, general_labels):
    generalized_ids = np.empty(0)
    for id in ids:
        generalized_ids = np.append(generalized_ids,
                                    general_labels.loc[general_labels['ApplianceID'] == id, 'GeneralID'].unique()[0])
    return generalized_ids


def detect_day_events(dataframe, labels, general_tagging_info):
    phases = ["Phase1", "Phase2"]
    events_start = np.empty(0)
    events_end = np.empty(0)
    time_ticks = np.empty((2, len(dataframe)))
    events = list()
    i = 0

    for phase in phases:
        dataframe[phase + "_eOn"] = 0
        dataframe[phase + "_eOff"] = 0
        power = dataframe[phase + "_actual"]
        time = dataframe[phase + "_time"]

        events_start_temp, events_end_temp = calculate_events(power)
        events.append(events_start_temp)
        events_start = np.append(events_start, events_start_temp)
        events_end = np.append(events_end, events_end_temp)
        time_ticks[i] = time
        i += 1

    events_start, events_end, event_labels = extract_labeled_spikes(events_start, events_end, time_ticks, labels)
    events_start, events_end, event_labels = remove_overlapping_events(events_start, events_end, event_labels)
    event_labels = label_generalization(event_labels, general_tagging_info)

    phase = "Phase1"
    for index in range(events_start.size):
        if events_start[index] in events[0]:
            phase = "Phase1"
        else:
            phase = "Phase2"

        dataframe.loc[events_start[index], phase + "_eOn"] = event_labels[index]
        dataframe.loc[events_end[index], phase + "_eOff"] = event_labels[index]

    return dataframe

### Edit dataframes to have events

In [15]:
general_labels = pd.read_csv(r".\dataset\AllTaggingInfo_generalized.csv")
for index in range(len(data)):
    data[index] = detect_day_events(data[index], all_tagging_info, general_labels)

In [16]:
print(data[0])

# Now we need to process the data, so that it's in a format that we can give to the model


### Extract spikes

In [10]:
def get_index(expected, id):
    for touple in expected:
        if touple[0] == id:
            return touple[1]

    return 0


def process_spike(real, imag, app, event_start, event_end, label):
    window = 69
    length = event_end - event_start + 1
    labels = np.empty((length // window) + 1)
    spike = np.empty((((length // window) + 1), 3, window))

    for index in range(spike.shape[0] - 1):
        spike_window = np.empty((3, window))
        spike_window[0] = real[(event_start + window * index):(event_start + (window * (index + 1)))]
        spike_window[1] = imag[(event_start + window * index):(event_start + (window * (index + 1)))]
        spike_window[2] = app[(event_start + window * index):(event_start + (window * (index + 1)))]

        spike[index] = spike_window
        labels[index] = label

    # Last window may be too short, so we add it here, making sure it's the correct length
    spike_window = np.empty((3, window))
    spike_window[0] = real[(event_end - window):event_end]
    spike_window[1] = imag[(event_end - window):event_end]
    spike_window[2] = app[(event_end - window):event_end]

    spike[spike.shape[0] - 1] = spike_window
    labels[spike.shape[0] - 1] = label

    return spike, labels


def extract_spikes(dataframe):
    spikes = np.empty((0, 3, 69))
    labels = np.empty(0)
    expected1 = list()
    expected2 = list()

    for index in range(len(dataframe)):
        row = dataframe.iloc[index]

        if row["Phase1_eOff"] != 0:
            start = get_index(expected1, row["Phase1_eOff"])
            if start == 0:
                continue
            expected1.remove((row["Phase1_eOff"], start))

            real = dataframe["Phase1_real"]
            imag = dataframe["Phase1_img"]
            app = dataframe["Phase1_app"]

            spike, spike_label = process_spike(real, imag, app, start, index, row["Phase1_eOff"])
            spikes = np.vstack([spikes, spike])
            labels = np.append(labels, spike_label)

        if row["Phase2_eOff"] != 0:
            start = get_index(expected2, row["Phase2_eOff"])
            if start == 0:
                continue
            expected2.remove((row["Phase2_eOff"], start))

            real = dataframe["Phase2_real"]
            imag = dataframe["Phase2_img"]
            app = dataframe["Phase2_app"]

            spike, spike_label = process_spike(real, imag, app, start, index, row["Phase2_eOff"])
            spikes = np.vstack([spikes, spike])
            labels = np.append(labels, spike_label)

        if row["Phase1_eOn"] != 0:
            expected1.append((row["Phase1_eOn"], index))

        if row["Phase2_eOn"] != 0:
            expected2.append((row["Phase2_eOn"], index))

    return spikes, labels

In [11]:
spikes = np.empty((0, 3, 69))
labels = np.empty(0)
for index in range(len(data)):
    day_spikes, day_labels = extract_spikes(data[index])
    spikes = np.vstack([spikes, day_spikes])
    labels = np.append(labels, day_labels)

In [12]:
final_spikes = spikes.reshape(spikes.shape[0], -1)
encoder = OneHotEncoder(sparse_output=False, categories="auto")
final_labels = encoder.fit_transform(np.array(labels).reshape(-1, 1))

In [13]:
def is_one_hot_encoded(label):
    return np.all((label == 0) | (label == 1)) and np.sum(label) == 1

for i, label in enumerate(final_labels):
    if not is_one_hot_encoded(label):
        print(f"Label {i} is not correctly one-hot encoded.")

In [14]:
print(final_spikes.shape)
print(final_labels.shape)

(2334, 207)
(2334, 12)


In [15]:
train_X, test_X, train_y, test_y = train_test_split(final_spikes, final_labels, train_size=0.7, shuffle=True)
train_X = train_X.reshape(train_X.shape[0], 69, 3)
test_X = test_X.reshape(test_X.shape[0], 69, 3)

In [16]:
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(1633, 69, 3)
(1633, 12)
(701, 69, 3)
(701, 12)


# Make the model

In [17]:
model = Sequential()
model.add(SeparableConv1D(filters=32, kernel_size=2, activation='relu', input_shape=(69, 3)))
model.add(SeparableConv1D(filters=32, kernel_size=3, activation='relu'))
model.add(SeparableConv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(12, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
model.fit(train_X, train_y, epochs=50, batch_size=32, validation_data=(test_X, test_y))

Epoch 1/50
52/52 [==============================] - 6s 28ms/step - loss: 1.3714 - accuracy: 0.5364 - val_loss: 1.2314 - val_accuracy: 0.5321
Epoch 2/50
52/52 [==============================] - 1s 24ms/step - loss: 1.1087 - accuracy: 0.6044 - val_loss: 1.2819 - val_accuracy: 0.2739
Epoch 3/50
52/52 [==============================] - 1s 19ms/step - loss: 1.0996 - accuracy: 0.5940 - val_loss: 0.9667 - val_accuracy: 0.5849
Epoch 4/50
52/52 [==============================] - 1s 19ms/step - loss: 0.8398 - accuracy: 0.7208 - val_loss: 0.7841 - val_accuracy: 0.6919
Epoch 5/50
52/52 [==============================] - 1s 19ms/step - loss: 0.6942 - accuracy: 0.7740 - val_loss: 0.6851 - val_accuracy: 0.8031
Epoch 6/50
52/52 [==============================] - 1s 20ms/step - loss: 0.5900 - accuracy: 0.8114 - val_loss: 0.5700 - val_accuracy: 0.8559
Epoch 7/50
52/52 [==============================] - 1s 20ms/step - loss: 0.5640 - accuracy: 0.8157 - val_loss: 0.6288 - val_accuracy: 0.7646
Epoch 8/50
52

# Test the model

### Get a day

In [19]:
file_path_volts1 = r".\dataset\H1\Tagged_Training_10_24_1351062001\LF1V.csv"
file_path_amps1 = r".\dataset\H1\Tagged_Training_10_24_1351062001\LF1I.csv"
file_path_time_ticks1 = r".\dataset\H1\Tagged_Training_10_24_1351062001\TimeTicks1.csv"

file_path_volts2 = r".\dataset\H1\Tagged_Training_10_24_1351062001\LF2V.csv"
file_path_amps2 = r".\dataset\H1\Tagged_Training_10_24_1351062001\LF2I.csv"
file_path_time_ticks2 = r".\dataset\H1\Tagged_Training_10_24_1351062001\TimeTicks2.csv"

LF1V = pd.read_csv(file_path_volts1)
LF1I = pd.read_csv(file_path_amps1)
time_ticks1 = pd.read_csv(file_path_time_ticks1)

LF2V = pd.read_csv(file_path_volts2)
LF2I = pd.read_csv(file_path_amps2)
time_ticks2 = pd.read_csv(file_path_time_ticks2)

In [20]:
LF1V = np.array(LF1V)
LF1I = np.array(LF1I)
LF1V = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF1V])
LF1I = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF1I])

LF2V = np.array(LF2V)
LF2I = np.array(LF2I)
LF2V = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF2V])
LF2I = np.array([[complex(y.replace("i", "j")) for y in x] for x in LF2I])

In [21]:
L1_P = LF1V * np.conjugate(LF1I)
L2_P = LF2V * np.conjugate(LF2I)

L1_ComplexPower = np.sum(L1_P, axis=1)
L2_ComplexPower = np.sum(L2_P, axis=1)

L1_real = np.real(L1_ComplexPower)
L1_imag = np.imag(L1_ComplexPower)
L1_app = np.abs(L1_ComplexPower)

L2_real = np.real(L2_ComplexPower)
L2_imag = np.imag(L2_ComplexPower)
L2_app = np.abs(L2_ComplexPower)

L1_Pf = np.cos(np.angle(L1_P[:, 0]))
L2_Pf = np.cos(np.angle(L2_P[:, 0]))

L1_actual_power = L1_real * L1_Pf
L2_actual_power = L2_real * L2_Pf

In [22]:
time_ticks1_datetime = pd.to_datetime(time_ticks1.iloc[:, 0], unit='s')
time_ticks1_np = np.array(time_ticks1)
TS1_real = np.array(time_ticks1_datetime)

time_ticks2_datetime = pd.to_datetime(time_ticks2.iloc[:, 0], unit='s')
time_ticks2_np = np.array(time_ticks2)
TS2_real = np.array(time_ticks2_datetime)

In [23]:
test = pd.DataFrame()
len = min(len(L1_actual_power), len(L2_actual_power))

test["Phase1_actual"] = L1_actual_power[:len]
test["Phase1_real"] = L1_real[:len]
test["Phase1_img"] = L1_imag[:len]
test["Phase1_app"] = L1_app[:len]
test["Phase1_time"] = time_ticks1_np[:len]

test["Phase2_actual"] = L2_actual_power[:len]
test["Phase2_real"] = L2_real[:len]
test["Phase2_img"] = L2_imag[:len]
test["Phase2_app"] = L2_app[:len]
test["Phase2_time"] = time_ticks2_np[:len]

In [24]:
phases = ["Phase1", "Phase2"]
events_start = np.empty(0)
events_end = np.empty(0)
events = list()

for phase in phases:
    test[phase + "_eOn"] = 0
    test[phase + "_eOff"] = 0
    power = test[phase + "_actual"]
    time = test[phase + "_time"]

    events_start_temp, events_end_temp = calculate_events(power)
    events.append(events_start_temp)
    events_start = np.append(events_start, events_start_temp)
    events_end = np.append(events_end, events_end_temp)

events_start, events_end, event_labels = remove_overlapping_events(events_start, events_end,
                                                                   np.empty(events_start.shape))

phase = "Phase1"
for index in range(events_start.size):
    if events_start[index] in events[0]:
        phase = "Phase1"
    else:
        phase = "Phase2"

    test.loc[events_start[index], phase + "_eOn"] = event_labels[index]
    test.loc[events_end[index], phase + "_eOff"] = event_labels[index]

C:\Users\test\AppData\Local\Temp\ipykernel_20940\3548915574.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '6.95285478554724e-310' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test.loc[events_start[index], phase + "_eOn"] = event_labels[index]
C:\Users\test\AppData\Local\Temp\ipykernel_20940\3548915574.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '6.95285478554724e-310' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test.loc[events_end[index], phase + "_eOff"] = event_labels[index]
C:\Users\test\AppData\Local\Temp\ipykernel_20940\3548915574.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5.601617329415202e-67' has dtype incompatible with int64, please explicitly cast to a 

In [25]:
def get_index(expected, id):
    for touple in expected:
        if touple[0] == id:
            return touple[1]

    return 0


def process_spike_verification(real, imag, app, event_start, event_end):
    window = 69
    length = event_end - event_start + 1
    spike = np.empty((((length // window) + 1), 3, window))

    for index in range(spike.shape[0] - 1):
        spike_window = np.empty((3, window))
        spike_window[0] = real[(event_start + window * index):(event_start + (window * (index + 1)))]
        spike_window[1] = imag[(event_start + window * index):(event_start + (window * (index + 1)))]
        spike_window[2] = app[(event_start + window * index):(event_start + (window * (index + 1)))]

        spike[index] = spike_window

    # Last window may be too short, so we add it here, making sure it's the correct length
    spike_window = np.empty((3, window))
    spike_window[0] = real[(event_end - window):event_end]
    spike_window[1] = imag[(event_end - window):event_end]
    spike_window[2] = app[(event_end - window):event_end]

    spike[spike.shape[0] - 1] = spike_window

    return spike

spikes = np.empty((0, 3, 69))
expected1 = list()
expected2 = list()

for index in range(test.shape[0]):
    row = test.iloc[index]

    if row["Phase1_eOff"] != 0:
        start = get_index(expected1, row["Phase1_eOff"])
        if start == 0:
            continue
        expected1.remove((row["Phase1_eOff"], start))

        real = test["Phase1_real"]
        imag = test["Phase1_img"]
        app = test["Phase1_app"]

        spike = process_spike_verification(real, imag, app, start, index)
        spikes = np.vstack([spikes, spike])

    if row["Phase2_eOff"] != 0:
        start = get_index(expected2, row["Phase2_eOff"])
        if start == 0:
            continue
        expected2.remove((row["Phase2_eOff"], start))

        real = test["Phase2_real"]
        imag = test["Phase2_img"]
        app = test["Phase2_app"]

        spike = process_spike_verification(real, imag, app, start, index)
        spikes = np.vstack([spikes, spike])

    if row["Phase1_eOn"] != 0:
        expected1.append((row["Phase1_eOn"], index))

    if row["Phase2_eOn"] != 0:
        expected2.append((row["Phase2_eOn"], index))

In [26]:
print(spikes.shape)

(5947, 3, 69)


In [27]:
final_spikes = spikes.reshape(spikes.shape[0], -1)
print(final_spikes.shape)

(5947, 207)


In [28]:
final_spikes = final_spikes.reshape(final_spikes.shape[0], 69, 3)
predictions = model.predict(final_spikes)

186/186 [==============================] - 2s 6ms/step


In [29]:
actual_pred = np.empty(predictions.shape[0])
for index in range(predictions.shape[0]):
    actual_pred[index] = np.argmax(predictions[index])
print(actual_pred.shape)
print(events_start.shape)

(5947,)
(78,)


In [30]:
window = 69
num_windows = np.empty(events_start.shape)
for index in range(events_start.size):
    length = events_end[index] - events_start[index] + 1
    num_windows[index] = (length // window) + 1

event_prediction = np.empty(0)
i = 0
for index in range(num_windows.size):
    cur_index = num_windows[index]
    # for prediction_index in range(i, i + int(cur_index)):
    event_prediction = np.append(event_prediction, mode(actual_pred[i : i + int(cur_index)]))
    i += 1

In [31]:
print(actual_pred.shape)
print(events_start.shape)
print(event_prediction.shape)

(5947,)
(78,)
(78,)


# Visualize test

In [32]:
L1 = test["Phase1_actual"]
L2 = test["Phase2_actual"]
print(L1)
print(L2)

0         324.192609
1         323.342048
2         321.893517
3         322.506852
4         322.487934
             ...    
518927    313.525123
518928    313.553162
518929    314.005817
518930    311.033037
518931    315.104527
Name: Phase1_actual, Length: 518932, dtype: float64
0         109.257317
1         110.044759
2         109.567434
3         109.933433
4         109.259499
             ...    
518927    110.240662
518928    109.480661
518929    109.279563
518930    109.741390
518931    109.377934
Name: Phase2_actual, Length: 518932, dtype: float64


In [ ]:
# Define a color map based on the number of unique labels
unique_labels = np.unique(event_prediction)
num_labels = unique_labels.size
color_map = cm.rainbow(np.linspace(0, 1, num_labels))

# Create a dictionary to map labels to colors
label_to_color = {label: color_map[i % num_labels] for i, label in enumerate(unique_labels)}

# Plot entire base power
# plt.plot(time, power, label=label)

# Plot the predictions with colors based on labels
for i, prediction in enumerate(events_start):
    label = event_prediction[i]  # Get the label for the current event
    color = label_to_color.get(label, color_map[0])  # Use the color map or default to label 0

    # Get the time and power data for the current event
    start_idx = window * i
    end_idx = start_idx + window
    event_time = time[start_idx:end_idx]
    event_power = power[start_idx:end_idx]
    
    if label == 0:
        label = "Nothing"
    else:
        label = general_labels.loc[general_labels['GeneralID'] == label, 'ApplianceName'].unique()[0]

    # Plot the event with the assigned color
    plt.plot(event_time, event_power, label=label, color=color)

# Add labels to the plot
plt.xlabel("Time")
plt.ylabel("Power")
plt.legend()
plt.show()